In [1]:
# === Problem 1: 一键运行版 ===
import os, json, csv, glob, heapq
from typing import Dict, List, Tuple, Optional

# ---------- 数据结构 ----------
class Node:
    __slots__ = ("Id","Op","BufId","Size","Type","Pipe","Cycles","Bufs")
    def __init__(self, Id:int, Op:str,
                 BufId:Optional[int]=None, Size:Optional[int]=None, Type:Optional[str]=None,
                 Pipe:Optional[str]=None, Cycles:Optional[int]=None, Bufs:Optional[List[int]]=None):
        self.Id = Id
        self.Op = Op or ""
        self.BufId = BufId if BufId in (None,"") else int(BufId)
        self.Size  = 0 if Size  in (None,"") else int(Size)
        self.Type  = Type
        self.Pipe  = Pipe
        self.Cycles= 0 if Cycles in (None,"") else int(Cycles)
        self.Bufs  = [] if Bufs is None else list(Bufs)

# ---------- 解析器 ----------
def parse_csv(nodes_csv:str, edges_csv:str) -> Tuple[str, Dict[int,Node], Dict[int,List[int]], Dict[int,List[int]]]:
    nodes: Dict[int,Node] = {}
    with open(nodes_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            Id = int(row["Id"])
            nodes[Id] = Node(
                Id=Id, Op=row.get("Op",""),
                BufId=(None if row.get("BufId") in (None,"") else int(row["BufId"])),
                Size=(0 if row.get("Size") in (None,"") else int(row["Size"])),
                Type=(row.get("Type") or None),
                Pipe=(row.get("Pipe") or None),
                Cycles=(0 if row.get("Cycles") in (None,"") else int(row["Cycles"])),
                Bufs=[]
            )
    adj, radj = {i:[] for i in nodes}, {i:[] for i in nodes}
    with open(edges_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            u, v = int(row["StartNodeId"]), int(row["EndNodeId"])
            adj[u].append(v); radj[v].append(u)
    task_name = os.path.basename(nodes_csv).replace("_Nodes.csv","")
    return task_name, nodes, adj, radj

# ---------- 调度器 ----------
class PeakLiveScheduler:
    def __init__(self, nodes, adj, radj):
        self.nodes, self.adj, self.radj = nodes, adj, radj
        self.N = len(nodes)
        self.indeg = {i: len(radj[i]) for i in nodes}

    def _soon_free_bytes_if_do(self, u, rem_in):
        s = 0
        for v in self.adj[u]:
            nd = self.nodes[v]
            if nd.Op.upper()=="FREE" and rem_in[v]==1:
                s += nd.Size
        return s

    def _score(self, u, rem_in):
        nd = self.nodes[u]; op = nd.Op.upper()
        if op=="FREE":
            return (0, -nd.Size, u)
        elif op=="ALLOC":
            return (2, nd.Size, u)
        else:
            return (1, -self._soon_free_bytes_if_do(u, rem_in), u)

    def schedule(self):
        rem_in = dict(self.indeg)
        heap, inheap = [], set()
        for u,deg in rem_in.items():
            if deg==0:
                heapq.heappush(heap, (self._score(u, rem_in), u)); inheap.add(u)
        live, peak, order = 0, 0, []
        while heap:
            _, u = heapq.heappop(heap)
            if rem_in[u]!=0: continue
            inheap.discard(u)
            order.append(u)
            op = self.nodes[u].Op.upper()
            if op=="ALLOC":
                live += self.nodes[u].Size; peak = max(peak, live)
            elif op=="FREE":
                live -= self.nodes[u].Size; live = max(live,0)
            for v in self.adj[u]:
                rem_in[v]-=1
                if rem_in[v]==0 and v not in inheap:
                    heapq.heappush(heap,(self._score(v,rem_in),v)); inheap.add(v)
        return order, peak

# ---------- 输出 ----------
def write_schedule(task_name, schedule, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{task_name}_schedule.txt")
    with open(out_path,"w",encoding="utf-8") as f:
        for nid in schedule: f.write(f"{nid}\n")
    return out_path

def run_task_csv(nodes_csv, edges_csv, out_dir):
    task_name, nodes, adj, radj = parse_csv(nodes_csv, edges_csv)
    sch = PeakLiveScheduler(nodes, adj, radj)
    order, peak = sch.schedule()
    out_file = write_schedule(task_name, order, out_dir)
    return {"task":task_name,"nodes":len(nodes),"edges":sum(len(v) for v in adj.values()),
            "peak_live":peak,"output":out_file}

def find_and_run_all(base_dir, out_dir):
    results=[]
    nodes_paths=glob.glob(os.path.join(base_dir,"*_Nodes.csv"))
    for npth in nodes_paths:
        name=os.path.basename(npth).replace("_Nodes.csv","")
        epth=os.path.join(base_dir,f"{name}_Edges.csv")
        if os.path.exists(epth):
            try: results.append(run_task_csv(npth,epth,out_dir))
            except Exception as e: results.append({"task":name,"error":str(e)})
    return results,out_dir

# ---------- 评估 ----------
import pandas as pd
def evaluate_all(base_dir, schedule_dir, save_csv_path):
    def _parse_nodes(nodes_csv):
        nodes={}
        with open(nodes_csv,"r",encoding="utf-8") as f:
            reader=csv.DictReader(f)
            for row in reader:
                nid=int(row["Id"])
                nodes[nid]={"Op":row.get("Op",""),
                            "Size":int(row["Size"]) if row.get("Size") not in (None,"") else 0}
        return nodes
    def _parse_edges(edges_csv):
        from collections import defaultdict,Counter
        adj=defaultdict(list); indeg=Counter()
        with open(edges_csv,"r",encoding="utf-8") as f:
            reader=csv.DictReader(f)
            for row in reader:
                u,v=int(row["StartNodeId"]),int(row["EndNodeId"])
                adj[u].append(v); indeg[v]+=1
        return dict(adj),dict(indeg)
    def _read_schedule(path):
        return [int(line.strip()) for line in open(path) if line.strip()]
    def _validate_topo(seq,adj):
        pos={nid:i for i,nid in enumerate(seq)}
        for u,vs in adj.items():
            for v in vs:
                if pos[u]>=pos[v]:
                    return False,f"Topo violation {u}->{v}"
        return True,""
    def _check_live(seq,nodes):
        live=0
        for i,nid in enumerate(seq):
            op=nodes[nid]["Op"].upper();sz=nodes[nid]["Size"]
            if op=="ALLOC": live+=sz
            elif op=="FREE": live-=sz
            if live<0: return False,f"negative at {i}"
        return True,""
    def _peak(seq,nodes):
        live=0;peak=0
        for nid in seq:
            op=nodes[nid]["Op"].upper();sz=nodes[nid]["Size"]
            if op=="ALLOC": live+=sz; peak=max(peak,live)
            elif op=="FREE": live-=sz
        return peak
    def _baseline(nodes,adj,indeg):
        import heapq
        rem=indeg.copy()
        for nid in nodes: rem.setdefault(nid,0)
        ready=[nid for nid,d in rem.items() if d==0]; heapq.heapify(ready)
        order=[]
        while ready:
            u=heapq.heappop(ready); order.append(u)
            for v in adj.get(u,[]): rem[v]-=1; 
            if rem[v]==0: heapq.heappush(ready,v)
        return order
    rows=[]
    for npth in glob.glob(os.path.join(base_dir,"*_Nodes.csv")):
        name=os.path.basename(npth).replace("_Nodes.csv","")
        epth=os.path.join(base_dir,f"{name}_Edges.csv")
        spth=os.path.join(schedule_dir,f"{name}_schedule.txt")
        if not (os.path.exists(epth) and os.path.exists(spth)): continue
        nodes=_parse_nodes(npth); adj,indeg=_parse_edges(epth); seq=_read_schedule(spth)
        ok_perm=(set(seq)==set(nodes.keys())); ok_topo,msg1=_validate_topo(seq,adj)
        ok_nonneg,msg2=_check_live(seq,nodes)
        peak_user=_peak(seq,nodes); peak_base=_peak(_baseline(nodes,adj,indeg),nodes)
        improve=(peak_base-peak_user)/peak_base if peak_base>0 else None
        rows.append({"task":name,"nodes":len(nodes),
                     "ok_perm":ok_perm,"ok_topo":ok_topo,"ok_nonneg":ok_nonneg,
                     "peak_user":peak_user,"peak_base":peak_base,
                     "improve_vs_base":improve,"note":msg1 or msg2})
    df=pd.DataFrame(rows); os.makedirs(os.path.dirname(save_csv_path),exist_ok=True)
    df.to_csv(save_csv_path,index=False); return df

# === 一键运行 ===
base_dir = r"./CSV"          # 输入目录（放 *_Nodes.csv, *_Edges.csv）
out_dir  = r"./Problem1" # 输出目录（生成 *_schedule.txt 和评估表）

# 1) 批量调度
results, _ = find_and_run_all(base_dir, out_dir)

# 2) 评估
eval_csv = os.path.join(out_dir,"Problem1_eval_summary.csv")
df_eval = evaluate_all(base_dir, out_dir, eval_csv)

print(f"调度完成，评估表已输出到: {eval_csv}")
print(df_eval)


调度完成，评估表已输出到: ./Problem1\Problem1_eval_summary.csv
                   task  nodes  ok_perm  ok_topo  ok_nonneg  peak_user  \
0            Conv_Case0   2580     True     True       True      62778   
1            Conv_Case1  36086     True     True       True     476790   
2  FlashAttention_Case0   1716     True     True       True      42248   
3  FlashAttention_Case1   6952     True     True       True     171664   
4          Matmul_Case0   4160     True     True       True     131328   
5          Matmul_Case1  30976     True     True       True    1048832   

   peak_base  improve_vs_base note  
0     150807         0.583720       
1     570168         0.163773       
2      64264         0.342587       
3     248976         0.310520       
4     163840         0.198437       
5    1179648         0.110894       


In [2]:
import pandas as pd
import heapq
from collections import defaultdict

# 文件所在目录路径
file_dir = r'.\CSV版本'

# --------------------------
# 第一部分：数据预处理（不变）
# --------------------------
print("开始数据预处理...")

# 读取节点文件和边文件
nodes_df = pd.read_csv(f'{file_dir}\\Matmul_Case0_Nodes.csv')
edges_df = pd.read_csv(f'{file_dir}\\Matmul_Case0_Edges.csv')

# 1. 构建节点属性表（保留Cycles字段）
node_attributes = {}
for _, row in nodes_df.iterrows():
    node_id = int(row['Id'])
    op_type = row['Op']
    buf_id = int(row['BufId']) if pd.notna(row['BufId']) else None
    size = int(row['Size']) if pd.notna(row['Size']) else None
    cache_type = row['Type'] if pd.notna(row['Type']) else None
    cycles = int(row['Cycles']) if pd.notna(row['Cycles']) else 0
    node_attributes[node_id] = {
        'id': node_id,
        'op': op_type,
        'buf_id': buf_id,
        'size': size,
        'cache_type': cache_type,
        'cycles': cycles,
        'predecessors': [],
        'successors': []
    }

# 2. 处理边信息（不变）
in_degree = defaultdict(int)
for _, row in edges_df.iterrows():
    src = int(row['StartNodeId'])
    dst = int(row['EndNodeId'])
    node_attributes[dst]['predecessors'].append(src)
    node_attributes[src]['successors'].append(dst)
    in_degree[dst] += 1
    if src not in in_degree:
        in_degree[src] = 0

# 3. 构建缓冲区生命周期映射（不变）
buf_lifecycle = defaultdict(dict)
for node_id, attr in node_attributes.items():
    if attr['op'] == 'ALLOC' and attr['buf_id'] is not None:
        buf_id = attr['buf_id']
        buf_lifecycle[buf_id]['alloc'] = node_id
        buf_lifecycle[buf_id]['uses'] = []
    elif attr['op'] == 'FREE' and attr['buf_id'] is not None:
        buf_id = attr['buf_id']
        buf_lifecycle[buf_id]['free'] = node_id

# 填充使用节点（不变）
for buf_id, lifecycle in buf_lifecycle.items():
    if 'alloc' not in lifecycle or 'free' not in lifecycle:
        continue
    alloc_id = lifecycle['alloc']
    free_id = lifecycle['free']
    visited = set()
    queue = [alloc_id]
    while queue:
        current = queue.pop(0)
        if current == free_id:
            break
        if current in visited:
            continue
        visited.add(current)
        if current != alloc_id:
            lifecycle['uses'].append(current)
        for succ in node_attributes[current]['successors']:
            if succ not in visited:
                queue.append(succ)

# 4. 保存预处理结果（不变）
node_attr_df = pd.DataFrame(node_attributes.values())
node_attr_df.to_csv(f'{file_dir}\\Matmul_Case0_node_attributes.csv', index=False)

in_degree_df = pd.DataFrame(list(in_degree.items()), columns=['NodeId', 'InDegree'])
in_degree_df.to_csv(f'{file_dir}\\Matmul_Case0_in_degree.csv', index=False)

buf_lifecycle_list = []
for buf_id, data in buf_lifecycle.items():
    record = {
        'BufId': buf_id,
        'AllocNode': data['alloc'],
        'UseNodes': ','.join(str(x) for x in data['uses']) if data['uses'] else '',
        'FreeNode': data['free']
    }
    buf_lifecycle_list.append(record)
buf_lifecycle_df = pd.DataFrame(buf_lifecycle_list)
buf_lifecycle_df.to_csv(f'{file_dir}\\Matmul_Case0_buf_lifecycle.csv', index=False)

print('预处理完成，结果保存到以下CSV文件：')
print(f'{file_dir}\\Matmul_Case0_node_attributes.csv')
print(f'{file_dir}\\Matmul_Case0_in_degree.csv')
print(f'{file_dir}\\Matmul_Case0_buf_lifecycle.csv')

# --------------------------
# 第二部分：最终修复版调度序列生成
# --------------------------
print("\n开始生成最终修复版调度序列...")

# 1. 加载预处理数据（不变）
node_attr_df = pd.read_csv(f'{file_dir}\\Matmul_Case0_node_attributes.csv')
node_attributes = {}
for _, row in node_attr_df.iterrows():
    node_id = int(row['id'])
    node_attributes[node_id] = {
        'id': node_id,
        'op': row['op'],
        'buf_id': int(row['buf_id']) if pd.notna(row['buf_id']) else None,
        'size': int(row['size']) if pd.notna(row['size']) else None,
        'cache_type': row['cache_type'] if pd.notna(row['cache_type']) else None,
        'cycles': int(row['cycles']) if pd.notna(row['cycles']) else 0,
        'predecessors': eval(row['predecessors']),
        'successors': eval(row['successors'])
    }

in_degree_df = pd.read_csv(f'{file_dir}\\Matmul_Case0_in_degree.csv')
in_degree = {int(row['NodeId']): int(row['InDegree']) for _, row in in_degree_df.iterrows()}

buf_lifecycle_df = pd.read_csv(f'{file_dir}\\Matmul_Case0_buf_lifecycle.csv')
buf_lifecycle = {}
for _, row in buf_lifecycle_df.iterrows():
    buf_id = int(row['BufId'])
    use_nodes_str = str(row['UseNodes']) if pd.notna(row['UseNodes']) else ''
    uses = []
    if use_nodes_str and use_nodes_str != 'nan' and use_nodes_str != '0':
        uses = list(map(int, use_nodes_str.split(',')))
    buf_lifecycle[buf_id] = {
        'alloc': int(row['AllocNode']),
        'uses': uses,
        'free': int(row['FreeNode'])
    }

# 2. 初始化调度数据结构（新增节点状态跟踪）
schedule = []
candidate_heap = []
current_cache = 0
max_cache = 0
buf_in_use = set()
current_step = 0

# 新增1：节点状态标记（避免重复调度/重复加入）
node_status = {node_id: 'pending' for node_id in node_attributes}  # pending/processing/completed
retry_count = defaultdict(int)  # 节点重复加入候选池的次数（限制最大3次）
MAX_RETRY = 3  # 最大重试次数，超过则视为无效节点

# L0缓存进度跟踪（不变）
l0_progress = {
    'L0A': {'in_use': False, 'current_op': None, 'progress': 0},
    'L0B': {'in_use': False, 'current_op': None, 'progress': 0},
    'L0C': {'in_use': False, 'current_op': None, 'progress': 0}
}

# 未来需求队列参数（不变）
K = 15


# --------------------------
# 核心修复1：多维度优先级计算（不变）
# --------------------------
def calculate_priority(node, current_step):
    priority = 0
    buf_id = node['buf_id']
    op_type = node['op']

    if op_type == 'FREE' and buf_id is not None:
        buf_size = node['size'] if (node['size'] is not None and buf_id in buf_lifecycle) else 0
        priority += (100 + buf_size) * 0.4

    elif op_type in ['ALLOC', 'COPY_IN', 'COPY_OUT', 'MMAD', 'SOFTMAX'] and buf_id in buf_lifecycle:
        lifecycle = buf_lifecycle[buf_id]
        if 'free' in lifecycle and 'alloc' in lifecycle:
            total_lifecycle = lifecycle['free'] - lifecycle['alloc']
            if total_lifecycle > 0:
                current_reside = current_step - lifecycle['alloc']
                lifecycle_ratio = min(current_reside / total_lifecycle, 1.0)
                priority += lifecycle_ratio * 100 * 0.3

    if op_type not in ['ALLOC', 'FREE']:
        priority += node['cycles'] * 0.2

    if node['cache_type'] in ['L0A', 'L0B', 'L0C']:
        priority += 50 * 0.1

    return priority


# --------------------------
# 核心修复2：冲突预判函数（不变）
# --------------------------
def predict_cache_conflict(future_alloc_list, cache_type, current_cache_used):
    cache_capacity_map = {'L1': 4096, 'UB': 1024, 'L0A': 256, 'L0B': 256, 'L0C': 512}
    if cache_type not in cache_capacity_map:
        return False
    cache_capacity = cache_capacity_map[cache_type]
    future_total = sum(alloc['size'] for alloc in future_alloc_list if alloc['cache_type'] == cache_type)
    return (current_cache_used + future_total) > cache_capacity


# 初始化候选池（新增：仅加入pending状态的节点）
for node_id, degree in in_degree.items():
    if degree == 0 and node_status[node_id] == 'pending':
        node = node_attributes[node_id]
        priority = calculate_priority(node, current_step=0)
        heapq.heappush(candidate_heap, (-priority, node_id))
        node_status[node_id] = 'processing'  # 标记为处理中，避免重复加入


# --------------------------
# 核心修复3：调度主循环（解决无限循环）
# --------------------------
# 记录已处理的节点数量，用于判断是否所有节点都已调度
total_nodes = len(node_attributes)
while candidate_heap:
    # 终止条件1：所有节点已调度完成
    if len(schedule) >= total_nodes:
        print(f"提前终止：所有{total_nodes}个节点已调度完成")
        break

    current_step = len(schedule)

    # 步骤1：更新未来需求队列（修复：仅处理pending/processing节点）
    temp_heap = candidate_heap.copy()
    future_alloc_queue = []
    count = 0
    while temp_heap and count < K:
        try:
            neg_prio, node_id = heapq.heappop(temp_heap)
        except IndexError:
            break
        if node_status[node_id] != 'completed':  # 仅处理未完成的节点
            node = node_attributes[node_id]
            if node['op'] == 'ALLOC' and node['cache_type'] is not None and node['buf_id'] is not None:
                future_alloc_queue.append({
                    'buf_id': node['buf_id'],
                    'size': node['size'] if node['size'] is not None else 0,
                    'cache_type': node['cache_type']
                })
        count += 1

    # 步骤2：弹出当前最高优先级节点（修复：过滤已完成的节点）
    valid_node = False
    while candidate_heap and not valid_node:
        try:
            neg_prio, current_node_id = heapq.heappop(candidate_heap)
        except IndexError:
            break
        # 过滤条件：节点未完成且重试次数未超上限
        if node_status[current_node_id] == 'completed':
            continue
        if retry_count[current_node_id] > MAX_RETRY:
            print(f"节点{current_node_id}重试次数超上限（{MAX_RETRY}次），标记为无效")
            node_status[current_node_id] = 'completed'
            continue
        # 找到有效节点，退出过滤循环
        current_node = node_attributes[current_node_id]
        valid_node = True

    # 终止条件2：无有效节点可调度（堆非空但全是无效节点）
    if not valid_node:
        print(f"终止：候选池无有效节点可调度（已调度{len(schedule)}/{total_nodes}个节点）")
        break

    # 步骤3：L0缓存精细调度判断（修复：重试次数限制）
    l0_skip = False
    if current_node['op'] == 'ALLOC' and current_node['cache_type'] in ['L0A', 'L0B', 'L0C']:
        l0_type = current_node['cache_type']
        l0_info = l0_progress[l0_type]
        buf_id = current_node['buf_id']
        if buf_id is None:
            retry_count[current_node_id] += 1
            heapq.heappush(candidate_heap, (neg_prio, current_node_id))
            l0_skip = True
            continue

        if l0_info['in_use']:
            if l0_info['progress'] >= 90:
                free_node_id = next((n for n in node_attributes if
                                     node_attributes[n]['op'] == 'FREE' and
                                     node_attributes[n]['buf_id'] == buf_id and
                                     node_status[n] == 'pending'), None)  # 仅找pending的FREE节点
                if free_node_id and in_degree[free_node_id] == 0:
                    free_node = node_attributes[free_node_id]
                    free_prio = calculate_priority(free_node, current_step)
                    # 执行FREE节点调度
                    if buf_id in buf_in_use:
                        current_cache -= free_node['size'] if free_node['size'] is not None else 0
                        buf_in_use.remove(buf_id)
                    schedule.append(free_node_id)
                    node_status[free_node_id] = 'completed'  # 标记为完成
                    # 更新L0状态
                    l0_progress[l0_type]['in_use'] = False
                    l0_progress[l0_type]['current_op'] = None
                    l0_progress[l0_type]['progress'] = 0
                else:
                    retry_count[current_node_id] += 1
                    heapq.heappush(candidate_heap, (neg_prio, current_node_id))
                    l0_skip = True
            else:
                retry_count[current_node_id] += 1
                heapq.heappush(candidate_heap, (neg_prio, current_node_id))
                l0_skip = True
        else:
            if buf_id in buf_lifecycle:
                use_node_id = next((n for n in buf_lifecycle[buf_id].get('uses', [])
                                    if node_attributes[n]['op'] == 'MMAD' and
                                    node_status[n] == 'pending'), None)  # 仅找pending的使用节点
                if use_node_id:
                    l0_progress[l0_type]['in_use'] = True
                    l0_progress[l0_type]['current_op'] = use_node_id
    if l0_skip:
        continue

    # 步骤4：缓冲区冲突预判（修复：重试次数限制）
    alloc_skip = False
    if current_node['op'] == 'ALLOC' and current_node['cache_type'] is not None and current_node['buf_id'] is not None:
        cache_type = current_node['cache_type']
        buf_id = current_node['buf_id']
        buf_size = current_node['size'] if current_node['size'] is not None else 0

        # 计算当前该缓存已使用容量（修复：仅统计有效buf_id）
        current_cache_used = 0
        for bid in buf_in_use:
            if bid in buf_lifecycle and buf_lifecycle[bid].get('alloc') in node_attributes:
                alloc_node = node_attributes[buf_lifecycle[bid]['alloc']]
                if alloc_node['cache_type'] == cache_type and alloc_node['size'] is not None:
                    current_cache_used += alloc_node['size']

        if predict_cache_conflict(future_alloc_queue, cache_type, current_cache_used):
            # 筛选pending且入度为0的FREE节点
            free_node_candidates = [
                n for n in node_attributes
                if (node_attributes[n]['op'] == 'FREE' and
                    node_attributes[n]['cache_type'] == cache_type and
                    node_attributes[n]['buf_id'] is not None and
                    in_degree[n] == 0 and
                    node_status[n] == 'pending')
            ]

            if free_node_candidates:
                # 选择释放容量最大的FREE节点
                def get_free_size(node_id):
                    node = node_attributes[node_id]
                    return node['size'] if node['size'] is not None else 0

                free_node_id = max(free_node_candidates, key=get_free_size)
                free_node = node_attributes[free_node_id]
                free_buf_id = free_node['buf_id']

                # 释放缓存
                if free_buf_id in buf_in_use:
                    current_cache -= get_free_size(free_node_id)
                    buf_in_use.remove(free_buf_id)
                    if free_node['cache_type'] in ['L0A', 'L0B', 'L0C']:
                        l0_progress[free_node['cache_type']]['in_use'] = False
                        l0_progress[free_node['cache_type']]['current_op'] = None
                        l0_progress[free_node['cache_type']]['progress'] = 0

                # 调度FREE节点
                schedule.append(free_node_id)
                node_status[free_node_id] = 'completed'
                # 重新加入当前ALLOC节点（重试次数+1）
                retry_count[current_node_id] += 1
                heapq.heappush(candidate_heap, (neg_prio, current_node_id))
                alloc_skip = True
    if alloc_skip:
        continue

    # 步骤5：执行当前节点调度（标记为完成）
    schedule.append(current_node_id)
    node_status[current_node_id] = 'completed'  # 关键：调度后立即标记为完成，避免重复处理
    retry_count[current_node_id] = 0  # 重置重试次数

    # 步骤6：更新L0操作节点进度（不变）
    for l0_type in l0_progress:
        l0_info = l0_progress[l0_type]
        if l0_info['in_use'] and l0_info['current_op'] == current_node_id:
            step_progress = 100 / max(current_node['cycles'], 1)
            l0_info['progress'] = min(l0_info['progress'] + step_progress, 100)
            buf_id = current_node['buf_id']
            if buf_id is not None and buf_id in buf_lifecycle and 'free' in buf_lifecycle[buf_id]:
                free_node_id = buf_lifecycle[buf_id]['free']
                if free_node_id in node_attributes and in_degree[free_node_id] == 0 and node_status[free_node_id] == 'pending':
                    free_node = node_attributes[free_node_id]
                    free_prio = calculate_priority(free_node, current_step + 1)
                    heapq.heappush(candidate_heap, (-free_prio, free_node_id))
                    node_status[free_node_id] = 'processing'

    # 步骤7：更新缓存状态（不变）
    buf_id = current_node['buf_id']
    if current_node['op'] == 'ALLOC' and buf_id is not None:
        if buf_id not in buf_in_use:
            buf_size = current_node['size'] if current_node['size'] is not None else 0
            current_cache += buf_size
            buf_in_use.add(buf_id)

    elif current_node['op'] == 'FREE' and buf_id is not None:
        if buf_id in buf_in_use:
            buf_size = current_node['size'] if current_node['size'] is not None else 0
            current_cache -= buf_size
            buf_in_use.remove(buf_id)

    # 步骤8：更新最大缓存（不变）
    if current_cache > max_cache:
        max_cache = current_cache

    # 步骤9：更新后继节点入度（修复：仅加入pending节点）
    for succ_id in current_node['successors']:
        if succ_id not in node_attributes or node_status[succ_id] == 'completed':
            continue  # 跳过不存在或已完成的节点
        in_degree[succ_id] -= 1
        if in_degree[succ_id] == 0 and node_status[succ_id] == 'pending':
            succ_node = node_attributes[succ_id]
            succ_prio = calculate_priority(succ_node, current_step + 1)
            heapq.heappush(candidate_heap, (-succ_prio, succ_id))
            node_status[succ_id] = 'processing'  # 标记为处理中


# 4. 结果保存与输出（新增：校验调度完整性）
schedule_df = pd.DataFrame({'NodeId': schedule})
schedule_df.to_csv(f'{file_dir}\\Matmul_Case0_schedule_final.csv', index=False)

# 输出调度完整性信息
completed_nodes = set(schedule)
missing_nodes = [node_id for node_id in node_attributes if node_id not in completed_nodes]
print(f"\n最终调度完成！")
print(f"总节点数：{total_nodes} | 已调度节点数：{len(schedule)} | 未调度节点数：{len(missing_nodes)}")
if missing_nodes:
    print(f"未调度节点ID：{missing_nodes[:10]}...")  # 打印前10个未调度节点
print(f"最大缓存驻留容量: {max_cache}")
print(f"最终调度序列已保存至: {file_dir}\\Matmul_Case0_schedule_final.csv")
print("\n调度序列预览（前20个节点）:")
print(schedule[:20] if len(schedule) >= 20 else schedule)
print("\n调度序列预览（后20个节点）:")
print(schedule[-20:] if len(schedule) >= 20 else schedule)

开始数据预处理...
预处理完成，结果保存到以下CSV文件：
.\CSV版本\Matmul_Case0_node_attributes.csv
.\CSV版本\Matmul_Case0_in_degree.csv
.\CSV版本\Matmul_Case0_buf_lifecycle.csv

开始生成最终修复版调度序列...

最终调度完成！
总节点数：4160 | 已调度节点数：4160 | 未调度节点数：0
最大缓存驻留容量: 27904
最终调度序列已保存至: .\CSV版本\Matmul_Case0_schedule_final.csv

调度序列预览（前20个节点）:
[0, 5, 6, 14, 15, 23, 24, 32, 33, 41, 42, 50, 51, 59, 60, 68, 69, 74, 77, 78]

调度序列预览（后20个节点）:
[2933, 2935, 4035, 2934, 2936, 3146, 2937, 4156, 4157, 2938, 2940, 4038, 2939, 2941, 3149, 2942, 4158, 4159, 2943, 4143]
